# J/Psi Tuning Analysis (v1)

This notebook is dedicated to evaluated the training for the v1 tuning using EGAM2 2017 samples setted to have the same detection as the cut based in the same sample. Let's check:

- Best tuning configuration;
- Tuning efficiencies using the best configuration;
- Training plots;
- Roc curve;
- And the best tuning configuration;


## Import all necessary packages:

In [1]:
from saphyra.analysis import crossval_table
from Gaugi import load
import os, re, sys, glob
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import collections
%matplotlib inline
%config InlineBackend.figure_format='retina'

Welcome to JupyROOT 6.16/00
Using all sub packages with ROOT dependence


## Create the tuning file dict:

In [2]:
def create_op_dict(op):
    d = {
              op+'_pd_ref'    : "reference/"+op+"_cutbased/pd_ref#0",
              op+'_fa_ref'    : "reference/"+op+"_cutbased/fa_ref#0",
              op+'_sp_ref'    : "reference/"+op+"_cutbased/sp_ref",
              op+'_pd_val'    : "reference/"+op+"_cutbased/pd_val#0",
              op+'_fa_val'    : "reference/"+op+"_cutbased/fa_val#0",
              op+'_sp_val'    : "reference/"+op+"_cutbased/sp_val",
              op+'_pd_op'     : "reference/"+op+"_cutbased/pd_op#0",
              op+'_fa_op'     : "reference/"+op+"_cutbased/fa_op#0",
              op+'_sp_op'     : "reference/"+op+"_cutbased/sp_op",
            
              # Counts
              op+'_pd_ref_passed'    : "reference/"+op+"_cutbased/pd_ref#1",
              op+'_fa_ref_passed'    : "reference/"+op+"_cutbased/fa_ref#1",
              op+'_pd_ref_total'     : "reference/"+op+"_cutbased/pd_ref#2",
              op+'_fa_ref_total'     : "reference/"+op+"_cutbased/fa_ref#2",   
              op+'_pd_val_passed'    : "reference/"+op+"_cutbased/pd_val#1",
              op+'_fa_val_passed'    : "reference/"+op+"_cutbased/fa_val#1",
              op+'_pd_val_total'     : "reference/"+op+"_cutbased/pd_val#2",
              op+'_fa_val_total'     : "reference/"+op+"_cutbased/fa_val#2",  
              op+'_pd_op_passed'     : "reference/"+op+"_cutbased/pd_op#1",
              op+'_fa_op_passed'     : "reference/"+op+"_cutbased/fa_op#1",
              op+'_pd_op_total'      : "reference/"+op+"_cutbased/pd_op#2",
              op+'_fa_op_total'      : "reference/"+op+"_cutbased/fa_op#2",
    } 
    return d

tuned_info = collections.OrderedDict( {
              # validation
              "max_sp_val"      : 'summary/max_sp_val',
              "max_sp_pd_val"   : 'summary/max_sp_pd_val#0',
              "max_sp_fa_val"   : 'summary/max_sp_fa_val#0',
              # Operation
              "max_sp_op"       : 'summary/max_sp_op',
              "max_sp_pd_op"    : 'summary/max_sp_pd_op#0',
              "max_sp_fa_op"    : 'summary/max_sp_fa_op#0',
    
              #"loss"            : 'loss',
              #"val_loss"        : 'val_loss',
              #"accuracy"        : 'accuracy',
              #"val_accuracy"    : 'val_accuracy',
              #"max_sp_best_epoch_val": 'max_sp_best_epoch_val',
              } )

tuned_info.update(create_op_dict('tight'))
tuned_info.update(create_op_dict('medium'))
tuned_info.update(create_op_dict('loose'))
tuned_info.update(create_op_dict('vloose'))

## Open all tuning files:

In [3]:
cv = crossval_table( tuned_info )
cv.fill( '../tuning_data/jpsiee/v1/*/*/*.pic.gz', 'v1')
#!ls ../tuning_data/*

I0806 01:40:44.161090 140735925498752 macros.py:23] Reading file for v1 tag from ../tuning_data/jpsiee/v1/*/*/*.pic.gz
I0806 01:40:44.162466 140735925498752 macros.py:23] There are 1500 files for this task...
I0806 01:40:44.163589 140735925498752 macros.py:23] Filling the table... 
I0806 01:41:16.356154 140735925498752 macros.py:23] End of fill step, a pandas DataFrame was created...


2020-08-06 01:40:44,161 | Py.crossval_table                       INFO Reading file for v1 tag from ../tuning_data/jpsiee/v1/*/*/*.pic.gz
2020-08-06 01:40:44,162 | Py.crossval_table                       INFO There are 1500 files for this task...
2020-08-06 01:40:44,163 | Py.crossval_table                       INFO Filling the table... 
2020-08-06 01:41:16,356 | Py.crossval_table                       INFO End of fill step, a pandas DataFrame was created...


Let's keep only the best inits for each sort, configuration and eta/phi bin. To calculate this we must choose an evaluation method to keep the max value in each configuration. Here, we will get the best inists looking for the max SP value for each one.

In [4]:
best_inits = cv.filter_inits("max_sp_val")
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
0,v1,0,0,0,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.946916,0.951160,...,28455,205005,2815,2583,2846,20500,28149,32086,28455,205005
120,v1,0,0,0,1,3,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944001,0.956079,...,28455,205005,2815,3137,2846,20500,28149,32318,28455,205005
337,v1,0,0,0,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.941700,0.946240,...,28455,205005,2815,3523,2846,20500,28150,32230,28455,205005
197,v1,0,0,0,3,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944651,0.968377,...,28455,205005,2815,3143,2846,20500,28149,31584,28455,205005
297,v1,0,0,0,4,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944673,0.960998,...,28455,205005,2815,3174,2846,20500,28149,31779,28455,205005


## Fix the reference counts:

In [5]:
def fix_counts(table, op):
    table['%s_fa_ref_total'%op]  = table['%s_fa_op_total'%op]
    table['%s_pd_ref_total'%op]  = table['%s_pd_op_total'%op]
    table['%s_fa_ref_passed'%op] = table['%s_fa_ref'%op] * table['%s_fa_op_total'%op]
    table['%s_pd_ref_passed'%op] = table['%s_pd_ref'%op] * table['%s_pd_op_total'%op]
    
for op in ['tight','medium','loose','vloose']:
    fix_counts(best_inits, op)


In [6]:
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
0,v1,0,0,0,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.946916,0.951160,...,28455,205005,2815,2583,2846,20500,28149,32086,28455,205005
120,v1,0,0,0,1,3,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944001,0.956079,...,28455,205005,2815,3137,2846,20500,28149,32318,28455,205005
337,v1,0,0,0,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.941700,0.946240,...,28455,205005,2815,3523,2846,20500,28150,32230,28455,205005
197,v1,0,0,0,3,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944651,0.968377,...,28455,205005,2815,3143,2846,20500,28149,31584,28455,205005
297,v1,0,0,0,4,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944673,0.960998,...,28455,205005,2815,3174,2846,20500,28149,31779,28455,205005


## Plot the evolution for each configuration

Here, each configurations means the number of neurons in the hidden layer. The tuning proceeding follow this topology:

- First layer with 100 inputs;
- Second layer with 2 to 10 neurons (all with hyperbolic tangent);
- One output with hyperbolic tangent.

Let's check the SP, Fake and detection evolution for each layer configuration calculating the mean and std values for all 10 best inits (1 init per sort, 10 sorts, 10 values per configuration).

In [9]:

def plot_evolution( t , et_bin, eta_bin , output, display=False):

    sp_val_mean = []; sp_val_std = []
    for model_idx in t.model_idx.unique():
        table = t.loc[ (t.model_idx==model_idx) & (t.et_bin==et_bin) & (t.eta_bin==eta_bin)]    
        sp_val_mean.append( table['max_sp_val'].mean() * 100)
        sp_val_std.append( table['max_sp_val'].std() * 100)
    neurons = [ i+2 for i in range(len(sp_val_mean))]
    fig, ax = plt.subplots(1,1, figsize=(15,5))
    plt.errorbar(neurons, sp_val_mean, yerr=sp_val_std, label='Max SP (validation)')
    ax.set(xlabel='#neurons', ylabel='SP',
       title='Max SP in the validation set (et=%d, eta=%d)'%(et_bin,eta_bin))
    ax.grid()
    print('saving %s...'%output)
    plt.savefig(output)
    if display:
        plt.show()      
    else:
        plt.close(fig)

        
for et_bin in best_inits.et_bin.unique():
    for eta_bin in best_inits.eta_bin.unique():
        plot_evolution(best_inits, et_bin, eta_bin, 'sp_evolution_per_config_et%d_eta%d.pdf'%(et_bin,eta_bin))

saving sp_evolution_per_config_et0_eta0.pdf...
saving sp_evolution_per_config_et0_eta1.pdf...
saving sp_evolution_per_config_et0_eta2.pdf...
saving sp_evolution_per_config_et0_eta3.pdf...
saving sp_evolution_per_config_et0_eta4.pdf...
saving sp_evolution_per_config_et1_eta0.pdf...
saving sp_evolution_per_config_et1_eta1.pdf...
saving sp_evolution_per_config_et1_eta2.pdf...
saving sp_evolution_per_config_et1_eta3.pdf...
saving sp_evolution_per_config_et1_eta4.pdf...
saving sp_evolution_per_config_et2_eta0.pdf...
saving sp_evolution_per_config_et2_eta1.pdf...
saving sp_evolution_per_config_et2_eta2.pdf...
saving sp_evolution_per_config_et2_eta3.pdf...
saving sp_evolution_per_config_et2_eta4.pdf...


For simplicity, let's keep two neurons (model id = 0) in the hidden layer for all phase spaces.

## Calculate the cross validation table for 2 neurons for all phase spaces

In [15]:
best_inits = best_inits.loc[best_inits.model_idx==0]
best_inits.head()

,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
0,v1,0,0,0,0,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.946916,0.951160,...,28455,205005,2815,2583,2846,20500,28149,32086,28455,205005
120,v1,0,0,0,1,3,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944001,0.956079,...,28455,205005,2815,3137,2846,20500,28149,32318,28455,205005
337,v1,0,0,0,2,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.941700,0.946240,...,28455,205005,2815,3523,2846,20500,28150,32230,28455,205005
197,v1,0,0,0,3,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944651,0.968377,...,28455,205005,2815,3143,2846,20500,28149,31584,28455,205005
297,v1,0,0,0,4,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944673,0.960998,...,28455,205005,2815,3174,2846,20500,28149,31779,28455,205005


In [16]:
# 20 bins X 10 sorts = 200 rows
best_inits.shape

(150, 98)

In [17]:
cv.describe(best_inits)

,train_tag,et_bin,eta_bin,max_sp_val_mean,max_sp_val_std,max_sp_pd_val_mean,max_sp_pd_val_std,max_sp_fa_val_mean,max_sp_fa_val_std,max_sp_op_mean,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_total_mean,vloose_pd_val_total_std,vloose_fa_val_total_mean,vloose_fa_val_total_std,vloose_pd_op_total_mean,vloose_pd_op_total_std,vloose_fa_op_total_mean,vloose_fa_op_total_std
0,v1,0,0,0.945854,0.002518,0.958426,0.007121,0.066618,0.005874,0.944333,...,28455,205005,2845.5,0.527046,20500.5,0.527046,28455.0,0.0,205005.0,0.0
1,v1,0,1,0.932883,0.004367,0.945623,0.010756,0.079739,0.007228,0.930188,...,8736,171335,873.6,0.516398,17133.5,0.527046,8736.0,0.0,171335.0,0.0
2,v1,0,2,0.890077,0.020242,0.905878,0.035211,0.125238,0.030917,0.870265,...,499,27116,49.9,0.316228,2711.6,0.516398,499.0,0.0,27116.0,0.0
3,v1,0,3,0.917786,0.005463,0.944560,0.009227,0.108573,0.009881,0.914631,...,6656,229486,665.6,0.516398,22948.6,0.516398,6656.0,0.0,229486.0,0.0
4,v1,0,4,0.922870,0.025178,0.974074,0.035136,0.126395,0.047350,0.895782,...,272,16670,27.2,0.421637,1667.0,0.000000,272.0,0.0,16670.0,0.0
5,v1,1,0,0.954745,0.003550,0.962228,0.007757,0.052697,0.003196,0.953529,...,19141,96571,1914.1,0.316228,9657.1,0.316228,19141.0,0.0,96571.0,0.0
6,v1,1,1,0.945721,0.003757,0.957584,0.010709,0.066032,0.008145,0.942683,...,6436,73903,643.6,0.516398,7390.3,0.483046,6436.0,0.0,73903.0,0.0
7,v1,1,2,0.916555,0.016507,0.952525,0.044574,0.118212,0.026626,0.891054,...,442,23534,44.2,0.421637,2353.4,0.516398,442.0,0.0,23534.0,0.0
8,v1,1,3,0.941616,0.004816,0.962220,0.011527,0.078721,0.009936,0.936206,...,3865,127374,386.5,0.527046,12737.4,0.516398,3865.0,0.0,127374.0,0.0
9,v1,1,4,0.943226,0.015175,0.975000,0.026352,0.087706,0.032985,0.918510,...,201,13340,20.1,0.316228,1334.0,0.000000,201.0,0.0,13340.0,0.0


## Create the beamer table

In [20]:
# Create beamer presentation
for op in ['tight','medium','loose','vloose']:
    cv.dump_beamer_table( best_inits ,  [0.0, 7.0, 10.0, 15.0], 
                     [0, 0.8 , 1.37, 1.54, 2.37, 2.5], 
                     [op],
                     'tuning_v1_'+op,
                     title = op+' Tunings (v1) J/Psi',
                     tags = ['v1'],
                    )


I0806 01:50:18.657526 140735925498752 BeamerAPI.py:475] Started creating beamer file tuning_v1_tight.pdf latex code...
I0806 01:50:24.183648 140735925498752 BeamerAPI.py:475] Started creating beamer file tuning_v1_medium.pdf latex code...
I0806 01:50:29.814810 140735925498752 BeamerAPI.py:475] Started creating beamer file tuning_v1_loose.pdf latex code...
I0806 01:50:37.206774 140735925498752 BeamerAPI.py:475] Started creating beamer file tuning_v1_vloose.pdf latex code...


2020-08-06 01:50:18,657 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_tight.pdf latex code...
2020-08-06 01:50:24,183 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_medium.pdf latex code...
2020-08-06 01:50:29,814 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_loose.pdf latex code...
2020-08-06 01:50:37,206 | Py.BeamerTexReportTemplate1             INFO Started creating beamer file tuning_v1_vloose.pdf latex code...


## Plot all Training for each sort

In [21]:

def plot_training_curves_for_each_sort(table, et_bin, eta_bin, output, display=False):
    
    table = table.loc[(table.et_bin==et_bin) & (table.eta_bin==eta_bin)] 
    nsorts = len(table.sort.unique())
    fig, ax = plt.subplots(nsorts,2, figsize=(15,20))
    fig.suptitle(r'Monitoring Train Plot - Et = %d, Eta = %d'%(et_bin,eta_bin), fontsize=15)
    for idx, sort in enumerate(table.sort.unique()):
        current_table = table.loc[table.sort==sort]
        path=current_table.file_name.values[0]
        history = load(path)['tunedData'][current_table.model_idx.values[0]]['history']
        best_epoch = history['max_sp_best_epoch_val'][-1]
        # Make the plot here
        ax[idx, 0].set_xlabel('Epochs')
        ax[idx, 0].set_ylabel('Loss')
        ax[idx, 0].plot(history['loss'], c='b', label='Train Step')
        ax[idx, 0].plot(history['val_loss'], c='r', label='Validation Step') 
        ax[idx, 0].axvline(x=best_epoch, c='k', label='Best epoch')
        ax[idx, 0].legend()
        ax[idx, 0].grid()
        ax[idx, 1].set_xlabel('Epochs')
        ax[idx, 1].set_ylabel('SP')
        ax[idx, 1].plot(history['max_sp_val'], c='r', label='Validation Step') 
        ax[idx, 1].axvline(x=best_epoch, c='k', label='Best epoch')
        ax[idx, 1].legend()
        ax[idx, 1].grid()
    
    print(output)
    plt.savefig(output)
    if display:
        plt.show()
    else:
        plt.close(fig)
    
for et_bin in best_inits.et_bin.unique():
    for eta_bin in best_inits.eta_bin.unique():
        plot_training_curves_for_each_sort(best_inits,et_bin, eta_bin, 'train_evolution_best_config_et%d_eta%d.pdf'%(et_bin,eta_bin))

train_evolution_best_config_et0_eta0.pdf
train_evolution_best_config_et0_eta1.pdf
train_evolution_best_config_et0_eta2.pdf
train_evolution_best_config_et0_eta3.pdf
train_evolution_best_config_et0_eta4.pdf
train_evolution_best_config_et1_eta0.pdf
train_evolution_best_config_et1_eta1.pdf
train_evolution_best_config_et1_eta2.pdf
train_evolution_best_config_et1_eta3.pdf
train_evolution_best_config_et1_eta4.pdf
train_evolution_best_config_et2_eta0.pdf
train_evolution_best_config_et2_eta1.pdf
train_evolution_best_config_et2_eta2.pdf
train_evolution_best_config_et2_eta3.pdf
train_evolution_best_config_et2_eta4.pdf


## Check the integrate value:

Here we will check the integrated value:

In [22]:
cv.integrate(best_inits, 'v1') * 100

,tight_pd_ref,tight_fa_ref,tight_pd_val,tight_fa_val,tight_pd_op,tight_fa_op,medium_pd_ref,medium_fa_ref,medium_pd_val,medium_fa_val,...,loose_pd_val,loose_fa_val,loose_pd_op,loose_fa_op,vloose_pd_ref,vloose_fa_ref,vloose_pd_val,vloose_fa_val,vloose_pd_op,vloose_fa_op
mean,9.777948e+01,36.905394,97.781343,10.298649,97.779104,10.495375,9.788300e+01,40.114282,97.887658,10.583616,...,98.035010,11.573268,98.017104,11.746490,98.455613,4.699215e+01,98.463073,12.786029,98.455706,13.029544
std,1.170278e-14,0.000000,0.011919,0.297305,0.000652,0.110137,1.170278e-14,0.000000,0.010128,0.314970,...,0.006294,0.381908,0.000590,0.137695,0.000000,5.851389e-15,0.007482,0.528095,0.000529,0.130901


## Get the Best sort for all phase spaces:


Here we will access the best sort for each phase space. We can use this information to export each tuning in each phase space to a unique file that will be used in the pileup adjustiment step (using prometheus framework). We will get the best sort with the highest max SP value for each phase space always.

In [24]:
best_sorts = cv.filter_sorts(best_inits, 'max_sp_val')
print(best_sorts.shape)
best_sorts.head(15)

(15, 98)


,train_tag,et_bin,eta_bin,model_idx,sort,init,file_name,tuned_idx,max_sp_val,max_sp_pd_val,...,vloose_pd_ref_total,vloose_fa_ref_total,vloose_pd_val_passed,vloose_fa_val_passed,vloose_pd_val_total,vloose_fa_val_total,vloose_pd_op_passed,vloose_fa_op_passed,vloose_pd_op_total,vloose_fa_op_total
237,v1,0,0,0,8,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.950797,0.965905,...,28455,205005,2814,2226,2845,20501,28149,31784,28455,205005
514,v1,0,1,0,0,2,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.942272,0.958810,...,8736,171335,858,2677,874,17134,8572,31466,8736,171335
964,v1,0,2,0,0,2,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.917131,0.940000,...,499,27116,44,279,50,2712,442,4694,499,27116
1537,v1,0,3,0,2,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.924618,0.944444,...,6656,229486,630,2371,666,22948,6292,27086,6656,229486
2037,v1,0,4,0,8,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.973122,1.000000,...,272,16670,26,87,27,1667,259,2810,272,16670
2662,v1,1,0,0,2,0,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.959501,0.964995,...,19141,96571,1901,1258,1914,9657,19009,14243,19141,96571
2705,v1,1,1,0,0,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.952876,0.968944,...,6436,73903,637,908,644,7390,6362,12947,6436,73903
3437,v1,1,2,0,3,2,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.944404,1.000000,...,442,23534,41,166,44,2354,404,2999,442,23534
3605,v1,1,3,0,0,4,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.949059,0.961240,...,3865,127374,374,1095,387,12737,3738,12708,3865,127374
4210,v1,1,4,0,9,1,/Users/jodafons/Desktop/phd_local/prometheus/a...,0,0.970544,1.000000,...,201,13340,19,77,20,1334,194,1742,201,13340


## Dump all best tunings:

Here we will dump all 15 configuration into a json file. We use the same neural netowrks for each operation point. Only the thresholds are variated. The threshold correction (alpha and beta) and files will be calculated/generated using the pileup correction tool.

In [25]:
def get_best_models( best_sorts , etbins, etabins):

    from saphyra.layers.RpLayer import RpLayer
    # Just to remove the keras dependence
    import tensorflow as tf
    model_from_json = tf.keras.models.model_from_json
    import json
    
    models = []

    for et_bin in range(len(etbins)-1):
        for eta_bin in range(len(etabins)-1):
            d_tuned = {}
            best = best_sorts.loc[(best_sorts.et_bin==et_bin) & (best_sorts.eta_bin==eta_bin)]
            best.head()
            tuned = load(best.file_name.values[0])['tunedData'][best.model_idx.values[0]]
            model = model_from_json( json.dumps(tuned['sequence'], separators=(',', ':')) , 
                                 custom_objects={'RpLayer':RpLayer} )
            
            model.set_weights( tuned['weights'] )
            model.pop()  #remove last activation
            # This is the only way to pop the last layer (sigmoid activation)
            #from tensorflow.keras.models import Model
            #new_model = Model(model.inputs, model.layers[-2].output)
            model.summary()
  
            d_tuned['model']    = model
            d_tuned['etBin']    = [etbins[et_bin], etbins[et_bin+1]]
            d_tuned['etaBin']   = [etabins[eta_bin], etabins[eta_bin+1]]
            d_tuned['etBinIdx'] = et_bin
            d_tuned['etaBinIdx']= eta_bin
            models.append(d_tuned)
    return models
            
        
    


etbins = [0.0, 7.0, 10.0, 15.0]
etabins = [0, 0.8 , 1.37, 1.54, 2.37, 2.5] 

models = get_best_models( best_sorts, etbins, etabins)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________


## Dump to onnx format:

We will dump all best models using dummy thresholds. This threshold should be rewrited using the pileup correction tool provided by the prometheus framework.


In [26]:
def convert_to_onnx_with_dummy_thresholds( models, name, version, signature, model_output_format , operation, output):

    import onnx
    import keras2onnx
    from ROOT import TEnv
    model_etmin_vec = []
    model_etmax_vec = []
    model_etamin_vec = []
    model_etamax_vec = []
    model_paths = []

    slopes = []
    offsets = []

    for model in models:

        model_etmin_vec.append( model['etBin'][0] )
        model_etmax_vec.append( model['etBin'][1] )
        model_etamin_vec.append( model['etaBin'][0] )
        model_etamax_vec.append( model['etaBin'][1] )

        etBinIdx = model['etBinIdx']
        etaBinIdx = model['etaBinIdx']

        # Conver keras to Onnx
        model['model'].summary()
        onnx_model = keras2onnx.convert_keras(model['model'], model['model'].name)

        onnx_model_name = model_output_format%( etBinIdx, etaBinIdx )
        model_paths.append( onnx_model_name +'.onnx')

        # Save onnx mode!
        onnx.save_model(onnx_model, onnx_model_name+'.onnx')

        model_json = model['model'].to_json()
        with open(onnx_model_name+".json", "w") as json_file:
            json_file.write(model_json)
            # saving the model weight separately
            model['model'].save_weights(onnx_model_name+".h5")
        
        slopes.append( 0.0 )
        offsets.append( 0.0 )


    def list_to_str( l ):
        s = str()
        for ll in l:
          s+=str(ll)+'; '
        return s[:-2]

    # Write the config file
    file = TEnv( 'ringer' )
    file.SetValue( "__name__", name )
    file.SetValue( "__version__", version )
    file.SetValue( "__operation__", operation )
    file.SetValue( "__signature__", signature )
    file.SetValue( "Model__size"  , str(len(models)) )
    file.SetValue( "Model__etmin" , list_to_str(model_etmin_vec) )
    file.SetValue( "Model__etmax" , list_to_str(model_etmax_vec) )
    file.SetValue( "Model__etamin", list_to_str(model_etamin_vec) )
    file.SetValue( "Model__etamax", list_to_str(model_etamax_vec) )
    file.SetValue( "Model__path"  , list_to_str( model_paths ) )
    file.SetValue( "Threshold__size"  , str(len(models)) )
    file.SetValue( "Threshold__etmin" , list_to_str(model_etmin_vec) )
    file.SetValue( "Threshold__etmax" , list_to_str(model_etmax_vec) )
    file.SetValue( "Threshold__etamin", list_to_str(model_etamin_vec) )
    file.SetValue( "Threshold__etamax", list_to_str(model_etamax_vec) )
    file.SetValue( "Threshold__slope" , list_to_str(slopes) )
    file.SetValue( "Threshold__offset", list_to_str(offsets) )
    file.SetValue( "Threshold__MaxAverageMu", 100)
    file.WriteFile(output)


    
for op in ['Tight','Medium','Loose','VeryLoose']:

    format = 'data17_13TeV_EGAM2_probes_lhmedium_EGAM7_vetolhvloose.model_v1.electron'+op+'.et%d_eta%d'
    output = "ElectronRinger%sTriggerConfig.conf"%op
    convert_to_onnx_with_dummy_thresholds( models, 'TrigL2_20200806_v1', 'v1', 'electron', format ,op ,output)

tf executing eager_mode: True
I0806 01:59:29.373703 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.375675 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:29.400477 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.411777 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:29.494470 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.500080 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:29.515496 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.518922 140735925498752 onnx_ex.py:64] The maximum opset needed by this model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________

I0806 01:59:29.566123 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.569246 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:29.601114 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.603962 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:29.618877 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.626754 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:29.653606 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.661439 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimizati

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________

The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:29.797276 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.801523 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:29.832159 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.836072 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:29.861217 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:29.873436 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:29.901137 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:29.903386 140735925498752 main.py:46] tf.keras model eager_

...

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________

tf executing eager_mode: True
I0806 01:59:32.439243 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:32.446002 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:32.460922 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:32.465342 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:32.495007 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:32.500969 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:32.510844 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:32.514150 140735925498752 onnx_ex.py:64] The maximum opset needed by this model

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________

I0806 01:59:32.636098 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:32.656471 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:32.665180 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:32.694269 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_mode: False
I0806 01:59:32.697302 140735925498752 main.py:46] tf.keras model eager_mode: False
The ONNX operator number change on the optimization: 10 -> 5
I0806 01:59:32.706130 140735925498752 topology.py:348] The ONNX operator number change on the optimization: 10 -> 5
W0806 01:59:32.709975 140735925498752 onnx_ex.py:64] The maximum opset needed by this model is only 9.
tf executing eager_mode: True
I0806 01:59:32.724833 140735925498752 main.py:44] tf executing eager_mode: True
tf.keras model eager_

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 2)                 202       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 3         
Total params: 205
Trainable params: 205
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
______________